In [55]:
import os, sys
print(os.listdir('../../'))
print(os.getcwd())
sys.path.append("../")
sys.path.append("../src/network")
sys.path.append("../src")
print(os.getcwd())
import torch
import glob
from torch import optim
import numpy as np
from src.data.tokenizer import Tokenizer
import string
import os, sys
import torch.utils.data as D
import cv2
import numpy as np
from torchvision import transforms
from src.network.gen_model.gen_model import GenModel_FC
from torchvision.transforms import v2
from random import choices
import kornia as K
import kornia.feature as KF
from kornia_moons.viz import *
from src.data.data_loader import RIMES_data
from src.data.reader import read_rimes
import random
import matplotlib.pyplot as plt
from src.network.losses import Loss
from src.network.sia_model.model import SiameseNetwork

['data', 'handwriting-recognition', 'htr_self_supervised', 'manuscript-dating-sn', 'mnist_pipeline', 'Paper-ext', 'self-supervised-mnist', 'Thesis', 'ws-siamese']
c:\Users\Lisa\Documents\GitHub\handwriting-recognition\src_semi
c:\Users\Lisa\Documents\GitHub\handwriting-recognition\src_semi


In [56]:
sia_oov_folder = "../../ws-siamese/data/iam/words/supervised_htr/"

sia_train_file = sia_oov_folder + "ground_truth_train_filtered.txt"
sia_oov_valid_file = sia_oov_folder + "ground_truth_valid_oov_filtered.txt"
sia_oov_test_file = sia_oov_folder + "ground_truth_test_oov_filtered.txt"



In [57]:
def read_sia_oov(file):
    """Get data paths and labels (with max_word_len) of images in folder."""
    unique_words = {}
    lens = {}

    with open(file) as data_file:
        lines = data_file.read().splitlines()

    for line in lines:
        line_split = line.split(' ')
                
        if len(line_split) > 2:
            img_path, gt_label = line_split[0], ''.join(line_split[1:])
        else:
            img_path, gt_label = line_split[0], line_split[1]
        
        if gt_label in unique_words.keys():
            unique_words[gt_label] += 1
        else:
            unique_words[gt_label] = 1
            
        
        if len(gt_label) in lens.keys():
                lens[len(gt_label)] += 1
        else:
            lens[len(gt_label)] = 1
                
            
    print(f"number of unique words: {len(unique_words.keys())}")

    lens = dict(sorted(lens.items()))

    print(f"Number of words per word length")
    for key in lens.keys():
        print(key, lens[key])

    return unique_words, lens
            

In [58]:
def read_htr(text_file):
    """Get data paths and labels (with max_word_len) of images in folder."""
    dataset = []
    wid_dict = {}
    lens = {}
    unique_words = {}

    with open(text_file, encoding='utf-8') as data_file:
        lines = data_file.read().splitlines()

    for line in lines:
        line_split = line.split(' ')
        wid = None

        if not set(string.digits).isdisjoint(set(line_split[-1])):
            wid = line_split[-1]
            line_split = line_split[:-1]
                
        if len(line_split) > 2:
            img_path, gt_label = line_split[0], ''.join(line_split[1:])
        else:
            img_path, gt_label = line_split[0], line_split[1]
                
        if gt_label in unique_words.keys():
            unique_words[gt_label] += 1
        else:
            unique_words[gt_label] = 1

        if len(gt_label) in lens.keys():
            lens[len(gt_label)] += 1
        else:
            lens[len(gt_label)] = 1
            
            img_path = img_path.replace("/", "\\")

            if wid is not None:
                if wid not in wid_dict.keys():
                    wid_dict[wid] = []
                
                wid_dict[wid].append((img_path, gt_label))

            # img_path = os.path.join(folder, partition, img_path)
            dataset.append((img_path, gt_label, wid))
    print(f"number of words: {len(dataset)}")
    print(f"number of wids: {len(wid_dict.keys())}")

    lens = dict(sorted(lens.items()))

    print(f"Number of words per word length")
    for key in lens.keys():
        print(key, lens[key])

    return dataset, wid_dict, unique_words

In [62]:
sia_train_words, sia_train_lens = read_sia_oov(sia_train_file)
sia_oov_valid_words, sia_oov_valid_lens = read_sia_oov(sia_oov_valid_file)
sia_oov_test_words, sia_oov_test_lens = read_sia_oov(sia_oov_test_file)

print(sia_oov_valid_words.keys())
print(sia_oov_test_words.keys())

sia_oov_words = set(list(sia_oov_valid_words.keys()) + list(sia_oov_test_words.keys()))
sia_words = set(list(sia_oov_valid_words.keys()) + list(sia_oov_test_words.keys()) + list(sia_train_words.keys()))
print("total number of oov words:", len(sia_oov_words))
print(len(sia_train_words.keys()))

number of unique words: 3161
Number of words per word length
2 6415
3 7706
4 5655
5 3798
6 2706
7 2440
number of unique words: 224
Number of words per word length
2 160
3 357
4 380
5 260
6 250
7 277
number of unique words: 220
Number of words per word length
2 4
3 241
4 404
5 184
6 231
7 129
dict_keys(['new', 'too', 'loud', 'back', 'writing', 'special', 'cannot', 'wonder', 'stage', 'At', 'man', 'best', 'Duke', 'express', 'years', 'goes', 'Council', 'against', 'ister', 'bundle', 'force', 'sewers', 'mass', 'among', 'Mauro', 'show', 'Richard', 'left', 'avert', 'minimum', 'each', 'odds', 'foreign', 'passed', 'Congo', 'Senator', 'Russia', 'Macleod', 'Tories', 'nothing', 'likely', 'three', 'want', 'social', 'Nuclear', 'capable', 'cinema', 'young', 'women', 'moved', 'mother', 'if', 'she', 'doubt', 'If', 'became', 'Holland', 'pitch', 'rise', 'failing', 'numbers', 'Hayward', 'garden', 'factor', 'dingy', 'such', 'hopes', 'accents', 'worn', 'Towards', 'Glasgow', 'Hood', 'firmly', 'able', 'General

In [67]:
htr_folder = "../data/iam/words/"
htr_train_data, htr_train_wids, htr_train_words = read_htr(htr_folder + "ground_truth_train_filtered.txt")
htr_valid_data, htr_valid_wids, htr_valid_words = read_htr(htr_folder + "ground_truth_valid_filtered.txt")
htr_test_data, htr_test_wids, htr_test_words = read_htr(htr_folder + "ground_truth_test_filtered.txt")

print(len(htr_train_words.keys()))



number of words: 6
number of wids: 1
Number of words per word length
2 4301
3 5306
4 4147
5 2841
6 2128
7 1829
number of words: 6
number of wids: 2
Number of words per word length
2 1281
3 1605
4 1264
5 808
6 637
7 517
number of words: 6
number of wids: 2
Number of words per word length
2 1028
3 1288
4 984
5 663
6 495
7 434
3539


In [73]:
htr_oov_train = []

for word in list(htr_train_words.keys()):
    if word not in sia_oov_words:
        if word not in list(sia_train_words.keys()):
            htr_oov_train.append(word)

print(len(set(htr_oov_train)))

htr_oov_valid = []

for word in list(htr_valid_words.keys()):
    if word not in sia_oov_words:
        if word not in list(sia_train_words.keys()):
            htr_oov_valid.append(word)

print(len(set(htr_oov_valid)))

htr_oov_test = []

for word in list(htr_test_words.keys()):
    if word not in sia_oov_words:
        if word not in list(sia_train_words.keys()):
            htr_oov_test.append(word)

print(len(set(htr_oov_test)))

htr_oov_test_oov = []

for word in set(htr_oov_test):
    if word not in htr_oov_train:
        if word not in set(htr_oov_train):
            htr_oov_test_oov.append(word)

print(len(set(htr_oov_test_oov)))
    

1501
549
419
276
